In [4]:
from openai import OpenAI
import pandas as pd
import os
import json
import logging
import time
import dotenv
import asyncio
from openai import AsyncOpenAI

dotenv.load_dotenv()

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("Tariff Analysis")


In [5]:
def read_transcript(wrds_path: str, file_path: str) -> str:
    transcript_path = os.path.join(wrds_path, file_path)
    with open(transcript_path, "r", encoding="utf-8", errors="replace") as f:
        return f.read()
    
metadata = pd.read_csv("/oak/stanford/groups/maggiori/GCAP/data/shared/WRDS_Transcripts/output/latest/metadata.csv")
metadata = metadata[(metadata["date"] >= "2025-01-01")]
df = metadata.sample(100).copy()
df['year'] = df.date.map(lambda x: x[:4])
df['file_path'] = df.apply(lambda x: x.year + "/" + str(x.key) + ".txt", axis=1)
wrds_path = "/oak/stanford/groups/maggiori/GCAP/data/shared/WRDS_Transcripts/output/latest/texts"
df['transcript'] = df.apply(lambda x: read_transcript(wrds_path, x.file_path), axis=1)
df.head()

,key,date,companyname_primary,companyid_primary,gvkey_primary,companynames,companyids,gvkeys,version_date,year,file_path,transcript
363719,1932760249,2025-03-13,Ampco-Pittsburgh Corporation,251148,[1613],['Ampco-Pittsburgh Corporation'],[251148],[[1613]],2025-04-23,2025,2025/1932760249.txt,"HEADLINE= Ampco-Pittsburgh Corporation, Q4 202..."
357571,1915256373,2025-02-07,"AptarGroup, Inc.",326230,[28176],"['AptarGroup, Inc.']",[326230],[[28176]],2025-04-01,2025,2025/1915256373.txt,"HEADLINE= AptarGroup, Inc., Q4 2024 Earnings C..."
357589,1915292596,2025-02-06,CapitaLand China Trust,30468171,[281750],['CapitaLand China Trust'],[30468171],[[281750]],2025-04-01,2025,2025/1915292596.txt,"HEADLINE= CapitaLand China Trust, 2024 Earning..."
362823,1932346959,2025-02-26,Kaldvik AS,275808956,[338642],['Kaldvik AS'],[275808956],[[338642]],2025-04-01,2025,2025/1932346959.txt,"HEADLINE= Kaldvik AS, Q4 2024 Earnings Call, F..."
366979,1939588291,2025-05-01,"Beazer Homes USA, Inc.",332730,[29755],"['Beazer Homes USA, Inc.']",[332730],[[29755]],2025-05-23,2025,2025/1939588291.txt,"HEADLINE= Beazer Homes USA, Inc., Q2 2025 Earn..."


#### Process single transcript

In [ ]:
# Modify OpenAI's API key and API base to use vLLM's API server.

openai_api_key = "None"
openai_api_base = "http://sh03-18n07:12345/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,

)

transcript = df.iloc[0]['transcript']

tariffs_sys_prompt = """ 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are assisting me in analyzing companies' earnings calls. The transcript of the earnings call will be supplied as the user prompt. 

## Response Instructions: Part 1 (Analysis) ##

The first part of your response should be an analysis of whether the firm is reporting that its decisions are being affected by tariff policies. Tariffs are defined as taxes imposed on imported foreign goods. These must be imposed by the importing country in order to be classified as tariffs. Tariffs are not export restrictions, quotas, embargoes, financial sanctions, boycotts, or non-tariff barriers.

This part of your response should be enclosed between the tags <ANALYSIS> and </ANALYSIS>. Keep your analysis to 300 words or less. Make sure to cover all of the following points in your summary analysis:

- Whether tariffs are discussed explicitly or implicitly (e.g., by not using the word "tariffs" but referencing impacts on the firm's business that clearly relate to tariffs).
- Whether the firm discusses current tariffs (i.e., tariffs that have already been imposed) or the potential of future tariffs (i.e., tariffs that have not yet been imposed).
- Whether the firm discusses tariffs on the goods it sells, or on those that it buys.
- The nature and details of the tariff policies that are discussed.
- The countries that are imposing the tariffs and those that are subject to the tariffs.
- Any impacts on the firms' current profits.
- Any potential impact on the firm's future profits.
- Any impacts on the firms' behavior (e.g., in terms of investment, pricing, employment, inventory, R&D, project delay, supply chains, or other future plans).
- Any details on the geographies affected by the changes in the firms' behavior (e.g., if the firm reports importing less, report from which destination).

## Response Instructions: Part 2 (Structured JSON Output) ##

The second part of your response should be a structured output in JSON format that recaps your analysis in a structured way. This part of your response should be enclosed between the tags <JSON> and </JSON>. The JSON output must have the following fields exactly. Please make sure to enforce the JSON schema specified below strictly: i.e., the column names should correspond exactly to those listed below:

1. A boolean flag called "tariffs_any", which should be 1 if the firm discusses tariffs at any point in the call, and 0 otherwise. Even if the term "tariffs" is not explicitly used throughout the call, you should return a 1 if the firm discusses impacts on its business that clearly relate to tariffs.
2. A field called "countries_imposing_tariffs", listing the countries whose tariffs policy the firm discusses, if any. For example, if the firm reports concerns about tariffs imposed by the US government on goods imported from China, this field should say "USA"
3. A field called "countries_receiving_tariffs", listing the entities (countries or firms) targeted by the tariffs discussed by the firm, if any. For example, if the firm reports concerns about tariffs imposed by the US government on goods imported from China, this field should say "China".
4. A boolean flag called "investment_up", which should be 1 if the firm says it is increasing or planning to increase its investment as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs.
5. A boolean flag called "investment_down", which should be 1 if the firm says it is decreasing or planning to decrease its investment as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs.
6. A field called "countries_investment_up", listing the countries where the firm reports increasing or planning to increase investment as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
7. A field called "countries_investment_down", listing the countries where the firm reports decreasing or planning to decrease investment as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
8. A boolean flag called "domestic_investment_up", which should be 1 if the firm says it is increasing or planning to increase its investment in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs. 
9. A boolean flag called "domestic_investment_down", which should be 1 if the firm says it is decreasing or planning to decrease its investment in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its investment decisions to tariffs. 
10. A boolean flag called "labor_up", which should be 1 if the firm says it is increasing or planning to increase its number of workers or their hours worked as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
11. A boolean flag called "labor_down", which should be 1 if the firm says it is decreasing or planning to decrease its number of workers or their hours worked as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
12. A field called "countries_labor_up", listing the countries where the firm reports increasing or planning to increase its number of workers or their hours worked as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
13. A field called "countries_labor_down", listing the countries where the firm reports decreasing or planning to decrease its number of workers or their hours worked as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
14. A boolean flag called "domestic_labor_up", which should be 1 if the firm says it is increasing or planning to increase its number of workers or their hours worked in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
15. A boolean flag called "domestic_labor_down", which should be 1 if the firm says it is decreasing or planning to decrease its number of workers or their hours worked in its headquarter country as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its employment decisions to tariffs.
16. A boolean flag called "sales_price_up", which should be 1 if the firm says it is increasing or planning to increase the price it charges for the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its product pricing decisions to tariffs. 
17. A boolean flag called "sales_price_down", which should be 1 if the firm says it is decreasing or planning to decrease the price it charges for the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its product pricing decisions to tariffs.
18. A field called "countries_sales_price_up", listing the countries where the firm reports increasing or planning to increase the price it charges for the products it sells as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
19. A field called "countries_sales_price_down", listing the countries where the firm reports decreasing or planning to decrease the price it charges for the products it sells as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
20. A boolean flag called "input_price_up", which should be 1 if the firm says it faces or expects to face higher prices on the inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its input prices to tariffs. 
21. A boolean flag called "input_price_down", which should be 1 if the firm says it faces or expects to face lower prices on the inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its input prices to tariffs.
22. A field called "countries_input_price_up", listing the countries where the firm reports facing or expecting to face higher prices on the inputs it purchases as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
23. A field called "countries_input_price_down", listing the countries where the firm reports facing or expecting to face lower prices on the inputs it purchases as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
24. A boolean flag called "inventory_product_up", which should be 1 if the firm says it is increasing or expecting to increase the amount of inventory of the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs. 
25. A boolean flag called "inventory_product_down", which should be 1 if the firm says it is decreasing or expecting to decrease the amount of inventory of the products it sells as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs.
26. A boolean flag called "inventory_input_up", which should be 1 if the firm says it is increasing or expecting to increase the amount of inventory of inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs. 
27. A boolean flag called "inventory_input_down", which should be 1 if the firm says it is decreasing or expecting to decrease the amount of inventory of inputs it purchases as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its inventory decisions to tariffs.
28. A boolean flag called "rd_up", which should be 1 if the firm says it is increasing or expecting to increase the amount of research and development it undertakes as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its research and development decisions to tariffs. 
29. A boolean flag called "rd_down", which should be 1 if the firm says it is decreasing or expecting to decrease the amount of research and development it undertakes as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its research and development decisions to tariffs.
30. A boolean flag called "delay", which should be 1 if the firm says it is delaying a decision as a result of current or future tariffs. Only set the flag to 1 if the company explicitly attributes the delay to tariffs.
31. A boolean flag called "supply_chain_adj", which should be 1 if the firm says it is adjusting its supply chain as a result of current or future tariffs. Only set the flag to 1 if the company is explicitly adjusting supply chains due to tariffs.
32. A field called "countries_supply_chain_adj_towards", listing the countries where the firm reports increasing the exposure of its supply chain as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
33. A field called "countries_supply_chain_adj_away", listing the countries where the firm reports reducing the exposure of its supply chain as a result of current or future tariffs. If no country is specified or it is unclear, report "NaN".
34. A boolean flag called "lobbying_up", which should be 1 if the firm says it is increasing or expecting to increase its lobbying activity as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its lobbying policy to tariffs. 
35. A boolean flag called "lobbying_down", which should be 1 if the firm says it is decreasing or expecting to decrease its lobbying activity as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its lobbying policy to tariffs.
36. A boolean flag called "profit_margin_up", which should be 1 if the firm says its profit margins have increased or are expected to increase as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its profit margins to tariffs. 
37. A boolean flag called "profit_margin_down", which should be 1 if the firm says its profit margins have decreased or are expected to decreased as a result of current or future tariffs and 0 otherwise. Only set the flag to 1 if the company explicitly attributes a change in its profit margins to tariffs.
38. A boolean flag called "financing_adj", which should be 1 if the firm says it is adjusting its financing decisions in response to current or future tariffs. Only set the flag to 1 if the company is explicitly adjusting its financing decisions due to tariffs.

## Response Instructions: Part 3 (Summary) ##
Write a single summary of 100 words or fewer that captures only how the firm is (or may be) affected by tariffs. Omit all unrelated content. If the firm is not affected by any tariffs, write "Not affected." Enclose this analysis between the tags <SUMMARY> and </SUMMARY>.

## Response Instructions: Part 4 (Evaluation) ##
The fourth part of your response should be an evaluation of how well the JSON structured summary agrees with your initial analysis. Keep the evaluation to 100 words or less. This part of your response should be enclosed between the tags <EVAL> and </EVAL> and be 100 words or less.

## Important Notes ##
- Do not consider restrictions on exports (imposed by the seller country) or generic financial sanctions as indications of tariffs. Tariffs must be taxes imposed by the importing country.
<|end_of_text|>
"""

tariffs_user_prompt = '''
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Analyze the tariffs information from the transcript below:
<|start_transcript_id|>
{}
<|end_transcript_id|>
<|end_of_text|> '''.format(transcript)

chat_response = client.chat.completions.create(
    model="/scratch/groups/maggiori/raw_model_weights/Llama-3.3-70B-Instruct-AWQ",
    messages=[
        {"role": "system", "content": tariffs_sys_prompt},
        {"role": "user", "content": tariffs_user_prompt},
    ],
    max_tokens=2048,
    temperature=0
)

llm_output = chat_response.choices[0].message.content
print("Chat response:\n", llm_output)

2025-07-28 10:38:09,361 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/chat/completions "HTTP/1.1 200 OK"


Chat response:
 <ANALYSIS>
The firm discusses tariffs explicitly in the transcript. They mention that the tariffs are not expected to have a significant direct impact on their business, but they may have indirect effects through changing demand dynamics involving customers in Canada and Mexico. The firm expects increased demand in the U.S. and stable demand in Europe for their roll business if tariffs are implemented. They also mention that many of their largest competitors for their Forged and Cast Engineered Products (FEP) business are outside the U.S., which could lead to increased demand for their FEP products.

The tariffs discussed are current and potential future tariffs. The firm mentions that mill rolls are not subject to tariffs, as they are classified as rolling mill components or parts, not as steel products like tool steel or rebar. The countries imposing tariffs are not explicitly mentioned, but it can be inferred that the U.S. is one of them, given the discussion about t

In [8]:
cols = [  
    "tariffs_any",
    "countries_imposing_tariffs",
    "countries_receiving_tariffs",
    "investment_up",
    "investment_down",
    "countries_investment_up",
    "countries_investment_down",
    "domestic_investment_up",
    "domestic_investment_down",
    "labor_up",
    "labor_down",
    "countries_labor_up",
    "countries_labor_down",
    "domestic_labor_up",
    "domestic_labor_down",
    "sales_price_up",
    "sales_price_down",
    "countries_sales_price_up",
    "countries_sales_price_down",
    "input_price_up",
    "input_price_down",
    "countries_input_price_up",
    "countries_input_price_down",
    "inventory_product_up",
    "inventory_product_down",
    "inventory_input_up",
    "inventory_input_down",
    "rd_up",
    "rd_down",
    "delay",
    "supply_chain_adj",
    "countries_supply_chain_adj_towards",
    "countries_supply_chain_adj_away",
    "lobbying_up",
    "lobbying_down",
    "profit_margin_up",
    "profit_margin_down",
    "financing_adj",
    "parse_error"]

# Function to safely parse JSON and handle errors
def safe_json_loads(output_str):
    try:
        result = {}
        parse_error = False
        # Remove the header/footer tags
        output_str = output_str.replace("<|start_header_id|>assistant<|end_header_id|>", "")
        output_str = output_str.replace("```json", "")
        output_str = output_str.replace("```", "</JSON>")
        
        # Extract JSON portion between <JSON> and </JSON>
        json_start = output_str.find("<JSON>") + len("<JSON>")
        json_end = output_str.find("</JSON>")
        if json_start == -1 or json_end == -1:
            parse_error = True
        else:
            json_str = output_str[json_start:json_end]
            result = json.loads(json_str)
        
        # Extract analysis text between <ANALYSIS> and </ANALYSIS>
        analysis_start = output_str.find("<ANALYSIS>") + len("<ANALYSIS>")
        analysis_end = output_str.find("</ANALYSIS>")
        if analysis_end == -1:
            analysis_end = output_str.find("<JSON>")
        if analysis_start == -1 or analysis_end == -1:
            parse_error = True
        else:
            result['analysis'] = output_str[analysis_start:analysis_end].strip()
            
        # Extract analysis text between <SUMMARY> and </SUMMARY>
        summary_start = output_str.find("<SUMMARY>") + len("<SUMMARY>")
        summary_end = output_str.find("</SUMMARY>")
        if summary_end == -1:
            summary_end = output_str.find("<EVAL>")
        if summary_start == -1 or summary_end == -1:
            parse_error = True
        else:
            result['summary'] = output_str[summary_start:summary_end].strip()
            
        eval_start = output_str.find("<EVAL>") + len("<EVAL>") 
        eval_end = output_str.find("</EVAL>")
        if eval_start == -1 or eval_end == -1:
            parse_error = True
        else:
            result['evaluation'] = output_str[eval_start:eval_end].strip()
        
        result['parse_error'] = "1" if parse_error else "0"
        
        return result
    except:
        return None
    
json_result = safe_json_loads(llm_output)
print(json.dumps(json_result, indent=2))

{
  "tariffs_any": 1,
  "countries_imposing_tariffs": "USA",
  "countries_receiving_tariffs": "Canada, Mexico",
  "investment_up": 0,
  "investment_down": 0,
  "countries_investment_up": "NaN",
  "countries_investment_down": "NaN",
  "domestic_investment_up": 0,
  "domestic_investment_down": 0,
  "labor_up": 0,
  "labor_down": 0,
  "countries_labor_up": "NaN",
  "countries_labor_down": "NaN",
  "domestic_labor_up": 0,
  "domestic_labor_down": 0,
  "sales_price_up": 0,
  "sales_price_down": 0,
  "countries_sales_price_up": "NaN",
  "countries_sales_price_down": "NaN",
  "input_price_up": 0,
  "input_price_down": 0,
  "countries_input_price_up": "NaN",
  "countries_input_price_down": "NaN",
  "inventory_product_up": 0,
  "inventory_product_down": 0,
  "inventory_input_up": 0,
  "inventory_input_down": 0,
  "rd_up": 0,
  "rd_down": 0,
  "delay": 0,
  "supply_chain_adj": 0,
  "countries_supply_chain_adj_towards": "NaN",
  "countries_supply_chain_adj_away": "NaN",
  "lobbying_up": 0,
  "lob

#### Batch processing with ASYNC

In [ ]:
def prepare_prompt(x):
    out =  f'''{tariffs_sys_prompt}\n<|begin_of_text|><|start_header_id|>user<|end_header_id|>
    Analyze the tariffs information from the transcript below:
    <|start_transcript_id|>{x}<|end_transcript_id|>
    <|end_of_text|> '''
    return out

prompts = df['transcript'].apply(prepare_prompt)

async_client = AsyncOpenAI(
    base_url="http://sh03-18n07:12345/v1",
    api_key="None"
)

async def process_text(prompt):
    try:
        response = await async_client.completions.create(
            model="/scratch/groups/maggiori/raw_model_weights/Llama-3.3-70B-Instruct-AWQ",
            prompt=prompt,
            max_tokens=2048,
            temperature=0,
            timeout=300
        )
        return response.choices[0].text if len(response.choices) > 0 else ""
    except Exception as e:
        print(f"Error: {e}")
        return ""

async def process_batch(prompts, batch_size = 10):
    results = []
    
    # Process in batches
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size]
        
        # Create tasks for concurrent execution
        tasks = [
            process_text(prompt) for prompt in batch
        ]
        
        # Execute batch concurrently and wait for all to complete
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)
        
        print(f"Completed batch {i//batch_size + 1}, processed {len(results)}/{len(prompts)} items")

    return results

results = await process_batch(list(prompts.values))

2025-07-28 10:42:49,324 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:51,375 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:51,752 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:53,329 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:53,466 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:55,028 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:55,858 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:56,561 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:42:57,518 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:02

Completed batch 1, processed 10/100 items


2025-07-28 10:43:53,307 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:53,331 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:54,205 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:55,868 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:57,085 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:57,426 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:57,908 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:43:58,677 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:44:00,299 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:44:02

Completed batch 2, processed 20/100 items


2025-07-28 10:45:01,703 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:03,159 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:03,650 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:03,754 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:04,279 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:05,331 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:05,420 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:06,924 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:10,420 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:45:10

...

Completed batch 8, processed 80/100 items


2025-07-28 10:51:35,661 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:36,302 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:36,555 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:36,995 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:38,556 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:40,412 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:40,930 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:43,291 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:44,782 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:51:46

Completed batch 9, processed 90/100 items


2025-07-28 10:52:44,824 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:47,341 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:49,858 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:49,972 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:50,554 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:52,888 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:53,337 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:53,457 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:52:55,048 - INFO - HTTP Request: POST http://sh04-06n05:12345/v1/completions "HTTP/1.1 200 OK"
2025-07-28 10:53:01

Completed batch 10, processed 100/100 items


In [11]:
df_tariffs = df.copy()
parsed_outputs = [safe_json_loads(output) for output in results]
all_dicts = []
for parsed_output in parsed_outputs:
    output_dict = {}
    for col in cols:
        output_dict[col] = str(parsed_output.get(col)) if isinstance(parsed_output, dict) else None
    if parsed_output is None:
        output_dict['parse_error'] = "1"
    all_dicts.append(output_dict)
    
for col in all_dicts[0].keys():
    df_tariffs[col] = [d[col] for d in all_dicts]
num_parse_error = df_tariffs[df_tariffs['parse_error'] == '1'].shape[0]
if num_parse_error > 0:
    print(f"Number of rows with parse error: {num_parse_error} ")
    
df_tariffs = df_tariffs[df_tariffs["tariffs_any"] == '1']
df_tariffs.head()

,key,date,companyname_primary,companyid_primary,gvkey_primary,companynames,companyids,gvkeys,version_date,year,...,delay,supply_chain_adj,countries_supply_chain_adj_towards,countries_supply_chain_adj_away,lobbying_up,lobbying_down,profit_margin_up,profit_margin_down,financing_adj,parse_error
363719,1932760249,2025-03-13,Ampco-Pittsburgh Corporation,251148,[1613],['Ampco-Pittsburgh Corporation'],[251148],[[1613]],2025-04-23,2025,...,0,1,USA,"Mexico, Canada",0,0,0,0,0,0
357571,1915256373,2025-02-07,"AptarGroup, Inc.",326230,[28176],"['AptarGroup, Inc.']",[326230],[[28176]],2025-04-01,2025,...,0,0,NaN,NaN,0,0,0,0,0,0
366979,1939588291,2025-05-01,"Beazer Homes USA, Inc.",332730,[29755],"['Beazer Homes USA, Inc.']",[332730],[[29755]],2025-05-23,2025,...,0,0,NaN,NaN,0,0,0,0,0,0
367918,1940492760,2025-05-06,"First Watch Restaurant Group, Inc.",1680518858,[39595],"['First Watch Restaurant Group, Inc.']",[1680518858],[[39595]],2025-05-23,2025,...,0,1,NaN,NaN,0,0,0,1,0,0
366891,1939562436,2025-05-08,"MKS Instruments, Inc.",398351,[119275],"['MKS Instruments, Inc.']",[398351],[[119275]],2025-05-23,2025,...,0,1,NaN,China,0,0,0,1,0,0
